Setting up a fancy stylesheet

In [1]:
from IPython.core.display import HTML
css_file = 'style.css'
HTML(open(css_file, 'r').read())

Setting up the required python &#8482; environment

In [2]:
#import numpy as np
import pandas as pd
from scipy.stats import bayes_mvs
#from math import factorial
import scikits.bootstrap as bs
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
sns.set_style('white')
sns.set_context('paper', font_scale = 2.0, rc = {'lines.linewidth': 1.5, 'figure.figsize' : (10, 8)})
%matplotlib inline
filterwarnings('ignore')

Importing the mock dataset and checking if it imported correctly

In [3]:
data = pd.read_csv('MOOC_Mock.csv')
data.head(3) #Looking at the first 3 rows

,File,Age,Gender,Delay,Stay,ICU,RVD,CD4,HR,Temp,CRP,WCC,HB,Rupture,Histo,Comp,MASS
0,1,38,Female,3,6,No,No,NaN,97,35.2,NaN,10.49,10.4,No,Yes,Yes,5
1,2,32,Male,6,10,No,Yes,57,109,38.8,45.3,7.08,19.8,No,No,Yes,8
2,3,19,Female,1,16,No,No,NaN,120,36.3,10.7,13.00,8.7,No,No,No,3


# Confidence intervals

## Introduction

<p>When conducting research by scrutinizing a sample set of individuals taken from a larger population, we create statistics.  We then use these statistics to infer parameter values for the greater population.  It can be used to guide patient treatment and management, by inferring that our results are applicable to that larger (patient) population.</p>
<p>However, there is no way that we can say with absolute confidence that our statistic is exactly what would to be found in the general population.  Instead we can place an arbitrary confidence around our test statistics, and infer that we are sure that the real (larger population) parameters are within that arbitrary confidence that we placed in our statistic(s).</p>

## Confidence interval around mean of a sample variable set

<p>Most textbooks begin with the chapter on confidence intervals (CI's) by using an example where the population mean and standard deviation is known.  This is almost never the case in healthcare research, so let's have a look at what is commonly observed.</p>
<p>Most of the time we are stuck with a set of variable values from our sample set and we can calculate the mean, standard deviation and standard error of that set.</p>
<p>In our example dataset we might look at the white cell count (WCC) on admission.  We could divide our sample patients into two groups, say those with and without retroviral disease.</p>
<p>In the code below we make two new DataFrames, with each DataFrame object attached to a new computer variable.<p>

In [4]:
RVD_neg = data[data['RVD'] == 'No'] # Only including the rows in the original DataFrame that have a Yes in the RVD column
RVD_neg_appx = RVD_neg[RVD_neg['Histo'] == 'Yes'] # Extracting only those who actually had appendicitis
RVD_pos = data[data['RVD'] == 'Yes'] # Doing the same but with No in the RVD column
RVD_pos_appx = RVD_pos[RVD_pos['Histo'] == 'Yes'] # Extracting only those who actually had appendicitis

<p>Let's look at the first rows in each new DataFrame using the *.head()* function.  Leaving the argument blank will result in a default of *5* rows.</p>

In [5]:
RVD_neg_appx.head()

,File,Age,Gender,Delay,Stay,ICU,RVD,CD4,HR,Temp,CRP,WCC,HB,Rupture,Histo,Comp,MASS
0,1,38,Female,3,6,No,No,NaN,97,35.2,NaN,10.49,10.4,No,Yes,Yes,5
7,8,19,Male,2,13,No,No,NaN,102,38.5,224.6,8.99,15.5,No,Yes,Yes,5
8,9,46,Male,2,9,No,No,NaN,107,36.2,385.1,10.26,15.5,No,Yes,No,1
9,10,19,Male,7,12,Yes,No,NaN,105,36.6,123,22.19,13.7,No,Yes,Yes,5
10,11,33,Male,2,14,No,No,NaN,104,36.8,NaN,16.20,12.7,No,Yes,No,9


In [6]:
RVD_pos_appx.head()

,File,Age,Gender,Delay,Stay,ICU,RVD,CD4,HR,Temp,CRP,WCC,HB,Rupture,Histo,Comp,MASS
4,5,28,Female,3,3,No,Yes,491,115,37.1,51.6,21.98,13.4,No,Yes,No,7
6,7,36,Male,6,14,No,Yes,203,95,37.5,NaN,16.21,17.7,Yes,Yes,Yes,7
19,20,33,Female,1,19,No,Yes,203,123,37.5,46.5,8.00,10.4,No,Yes,Yes,8
31,32,39,Male,2,10,No,Yes,102,100,36.8,208,20.43,12.9,Yes,Yes,No,1
32,33,33,Male,0,21,No,Yes,266,100,39.7,194.1,7.31,10.7,Yes,Yes,Yes,3


Note how the two **RVD** columns contain only *No* and *Yes* respectively.

<p>The easiest way to calculate the confidence intervals is using the *scipy.stats* function *bayes_mvs*.  It actually gives nine results, three rows of three values each.  The first row gives the mean and then the lower as well as the upper bound of the stated confidence interval.</p>
<p>The second row does the same but for the variance. The last row calculates the values for the standard deviation.</p>
<p>Let's first describe our two DataFrames then calculate the CI's.</p>

In [7]:
RVD_neg_appx['WCC'].describe() # Describing the WCC column in the RVD_neg DataFrame

count    78.000000
mean     14.044103
std       4.347939
min       6.610000
25%      11.270000
50%      13.655000
75%      16.305000
max      26.400000
Name: WCC, dtype: float64

In [8]:
RVD_pos_appx['WCC'].describe() # Describing the WCC column in the RVD_pos DataFrame

count    40.000000
mean     15.773250
std       4.934148
min       2.950000
25%      12.475000
50%      15.575000
75%      19.920000
max      24.890000
Name: WCC, dtype: float64

<p>Now for the CI's.  The *bayes_mvs()* function takes two arguments, the first is the values themselves and the second is the required confidence interval. In this example I want the 95% CI's.  I also make sure to discard all non-numerical values.</p>

In [9]:
bayes_mvs(RVD_neg_appx['WCC'].dropna(), 0.95)

(Mean(statistic=14.044102564102564, minmax=(13.063793818374444, 15.024411309830684)),
 Variance(statistic=19.408694495726504, minmax=(14.110883384251707, 26.648893927036038)),
 Std_dev(statistic=4.3908697719337093, minmax=(3.7564455785025959, 5.1622566700074142)))

In [10]:
bayes_mvs(RVD_pos_appx['WCC'].dropna(), 0.95)

(Mean(statistic=15.773250000000001, minmax=(14.195232891628422, 17.35126710837158)),
 Variance(statistic=25.661807500000002, minmax=(16.336646621396056, 40.140096800828552)),
 Std_dev(statistic=5.0316399443801929, minmax=(4.041861776631662, 6.33562126399839)))

<p>Let's put down in words what we have found from the calculations above.  We could state in a paper that there were 78 patients in the RVD negative group with acute appendicitis.  They had a mean WCC of 14.04 on admission, with a 95% confidence interval of 13.06 to 15.02.  There were 40 RVD positive patients with an average admission WCC of 15.77 (95% CI 14.20 to 17.35). (Note the mathematical rounding).</p>
<p>However, what do we mean by this 95% CI? Well, by our statistical anlaysis, we found a mean WCC of 15.77 in RVD positive patients.  If want want to infer this on the larger population we will have to say that according to our analysis we think that the true population admission WCC value out there for all RVD positive patients is between two values.  We can state those two values (lower and upper) with a set level of confidence.  Here, we are 95% confident that in the larger population out there, we think the true value lies between 14.20 and 17.35.</p>

<p>Let's drop our CI to 80%.  What do you expect to happen?  Well, the lower and upper values will be closer to the mean, making us less confident that our *prediction* (inference) is correct.  Let's see:</p>

In [11]:
bayes_mvs(RVD_pos_appx['WCC'], 0.8)

(Mean(statistic=15.773250000000001, minmax=(14.756206821106424, 16.790293178893577)),
 Variance(statistic=25.661807500000002, minmax=(18.742423588894685, 33.674780516226335)),
 Std_dev(statistic=5.0316399443801929, minmax=(4.329252081929936, 5.8029975457711798)))

Indeed, the values are now in a much narrower range of 14.76 to 16.79.  We are much less confident about this, though!

## What lies behind these calculations

<p>Let's start with the equation:</p>
$$ \overline { x } \pm t{ s }_{ SE } $$
<p>Now *t* is the t-statistic.  The t-statistic is calculated (or read from a table) and depends on the value of confidence desired (actually alpha value) and the degrees of freedom.</p>
<p>The *s<sub>SE</sub>* is the standard error of the mean or...</p>
$$ \frac { s }{ \sqrt { n }  }  $$
<p>...the standard deviation over the square root of the sample size.</p>
<p>Given the degrees of freedom and alpha value, a t-distribution is calculated.  It looks like a normal distribution and is different for any given parameters.  Therefore, when we state a confidence interval, we calculate what middle area will be covered by that confidence, i.e. we'll cover 0.95 of the total area under the curve and calculate where on the x-axis these cut-offs fall (the t-statistic).  This we multiply by the standard error of the mean, and subtract as well as add to the mean to get the lower and upper bounds of the required CI.  Neat!</p>

## Bootstrapping a confidence interval

<p>There is another method to calculate the CI.  It involves bootstrapping or resampling.  We are interested in infereing population parameters from statistics.  We would like to say that the real population parameter is between such and such a value and we are confident about this *guess* at a given level.</p>
<p>The only knowledge we have about the population, though, is our sample set.  Actually we have nothing else.  What we good do is take random samples from our sample set.  We need to take exactely the same number of random samples from our sample set.  In order not to have identical resamples, we allow a given sample value taken to be *thrown back* into the pool and have a chance of being resampled again, and again.  We can do thos thousanfs of times.  With the dawn of the age of fast computers, this is no problem.  Now we have as big a sample set as we want.  The hope is that this is a good refelction of the population.  From it we can form a distribution curve and calculate the CI<sup>s</sup>.</p>

In [12]:
bs.ci(RVD_pos_appx['WCC'].dropna(), alpha = 0.05, n_samples = 10000)
# Note that we state the alpha value directly and here I use 10000 resamples

array([ 14.23725,  17.23825])

<p>I can now given the CI<sup>s</sup> as the values in the array above.  Note, it will be different everytime you do it!</p>